In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import glob
import shutil
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [25]:
# Parent directory for saving outputs
base_local = Path("/home/ewan/THOR_output")
start = "2010-01-20T18:00:00"
end = "2010-01-21T03:30:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="gridrad_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="gridrad_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="gridrad", tags=["era5_pl", "era5_sl"], global_flow_margin=150, unique_global_flow=False)
option.check_options(track_options)
option.save_track_options(track_options, filename="gridrad_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["cell", "mcs"]
}
# visualize_options["middle_cloud"] = visualize.option.runtime_options(
#     "cell", save=True, style="presentation", figure_types=["mask"]
# )
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-08-23 20:09:57,210 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/gridrad_era5.yaml
2024-08-23 20:09:57,254 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/gridrad_geographic.yaml
2024-08-23 20:09:57,258 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-08-23 20:09:57,262 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/gridrad_mcs.yaml
2024-08-23 20:09:57,283 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [26]:
output_directory = base_local / "runs/gridrad_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/gridrad_demo",
)

2024-08-23 20:09:59,114 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-08-23 20:09:59,115 - thor.data.gridrad - DEBUG - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-08-23 20:09:59,116 - thor.data.gridrad - DEBUG - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-08-23 20:09:59,278 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-08-23 20:09:59,872 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-08-23 20:10:00,002 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-23 20:10:00,965 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-08-23 20:10:01,073 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-23 20:10:05,642 - thor.track - DEBUG - Processing hierarchy level 0.
2024-08-23 20:10:05,643 - thor.track - DEBUG - Tracking cell.
2024-08-23 20:10:09,353 - thor.match.match - DEBUG - Matching cell objects.
2024-08-23 20:10:09,366 

KeyboardInterrupt: 